# Transfer FGSM attack generator - Surrogate Model: MobileNetV2 (ImageNet)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Imports
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm

In [ ]:
mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [ ]:
def fgsm_video(input_path, output_path, epsilon=0.01, frames_to_attack=70):
  os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)

  cap = cv2.VideoCapture(input_path)
  if not cap.isOpened():
    raise RuntimeError("Cannot open video")

  fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
  W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_path, fourcc, float(fps), (W, H))
  if not out.isOpened():
    raise RuntimeError("VideoWriter failed to open")

  idx = 0
  while idx < frames_to_attack:
    ret, frame_bgr = cap.read()
    if not ret: break

    # model input prep (uses your preprocess)
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    inp = preprocess(tf.convert_to_tensor(frame_rgb, dtype=tf.float32))  # (1,Hm,Wm,3)

    # untargeted: use predicted class as label
    preds = pretrained_model(inp, training=False)
    pred_class = tf.argmax(preds, axis=-1)
    # print(preds.shape)
    target = tf.one_hot(pred_class, preds.shape[-1])

    # signed gradient (create_adversarial_pattern returns sign(grad) in your code)
    signed = create_adversarial_pattern(inp, target)  # (1,Hm,Wm,3)

    # resize signed grad -> original res, convert to pixel delta and add
    signed_resized = tf.image.resize(signed[0], (H, W))
    delta_pixels_rgb = signed_resized * (epsilon * 127.5)
    delta_bgr = cv2.cvtColor(delta_pixels_rgb.numpy().astype(np.float32), cv2.COLOR_RGB2BGR)

    adv = frame_bgr.astype(np.float32) + delta_bgr
    adv = np.clip(adv, 0, 255).astype(np.uint8)
    out.write(adv)

    idx += 1

  cap.release()
  out.release()

  try:
    tf.keras.backend.clear_session()
  except:
    pass

  return output_path

In [ ]:
# Src and Dst directories (Change it to your ideal directory)
src_dir = "/content/drive/MyDrive/faceforensics++/manipulated_sequences/Deepfakes/c40/videos/"
output_dir = "/content/drive/MyDrive/faceforensics++/Adversarial_attacked_sequences/TransferAttacks/FGSM/ImageNet/MobileNet/Deepfakes/"

In [ ]:
def FGSM(epsilon):
  if not os.path.exists(output_dir + "Epsilon" + str(epsilon) + "/"):
    os.makedirs(output_dir + "Epsilon" + str(epsilon) + "/")
  for video_path in tqdm(glob.glob(os.path.join(src_dir, "*.mp4"))):
    fname = os.path.basename(video_path)
    if os.path.exists(output_dir + "Epsilon" + str(epsilon) + "/" + fname):
      continue
    fgsm_video(src_dir + fname, output_dir + "Epsilon" + str(epsilon) + "/" + fname, epsilon)

In [ ]:
# Change the input to the epsilon values you want to study
FGSM(0.05)

  8%|▊         | 79/1000 [42:16<8:12:45, 32.10s/it]


KeyboardInterrupt: 